# Setup

In [18]:
from transformers import AutoTokenizer, AutoConfig
from torch import nn
from torch.nn import functional as F
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

import torch
import math

# Scaled Dot Product Attention

<figure style="text-align: center;">
  <img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098136789/files/assets/nlpt_0304.png" alt="Operations in scaled dot-product attention" width="600" height="89">
  <figcaption>Operations in scaled dot-product attention (Figure 3-4, <em>Natural Language Processing with Transformers</em>)</figcaption>
</figure>

In [2]:
text = """
First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.
"""

In [3]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [4]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2034,  6926,  1024,  2057,  2024, 14729,  3532,  4480,  1010,  1996,
         10717,  3619,  2204,  1012,  2054,  3691, 14175, 20175,  2015,  2006,
          2052, 15804,  2149,  1024,  2065,  2027,  2052, 10750,  2149,  2021,
          1996,  3565, 10258, 18518,  1010,  2096,  2009,  2020,  2878, 14045,
          1010,  2057,  2453,  3984,  2027,  7653,  2149, 23369,  2135,  1025,
          2021,  2027,  2228,  2057,  2024,  2205,  6203,  1024,  1996,  8155,
          2791,  2008, 21358, 29301,  2015,  2149,  1010,  1996,  4874,  1997,
          2256, 14624,  1010,  2003,  2004,  2019, 12612,  2000,  3327,  5562,
          2037, 14531,  1025,  2256,  9015,  6651,  2003,  1037,  5114,  2000,
          2068,  2292,  2149,  7195,  2023,  2007,  2256, 12694,  2015,  1010,
          9413,  2063,  2057,  2468, 26008,  2015,  1024,  2005,  1996,  5932,
          2113,  1045,  3713,  2023,  1999,  9012,  2005,  7852,  1010,  2025,
          1999, 21810,  2005,  7195,  1012]])

In [5]:
config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [6]:
config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [7]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.shape

torch.Size([1, 125, 768])

In [19]:
# Dot product
query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.einsum('ijk, ikl -> ijl', query, key.permute(0, 2, 1)) / math.sqrt(dim_k)
scores.size()

torch.Size([1, 125, 125])

In [22]:
weights = F.softmax(scores, dim=-1)
weights.shape, weights.sum(dim=-1).shape

(torch.Size([1, 125, 125]), torch.Size([1, 125]))

In [25]:
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]],
       grad_fn=<SumBackward1>)

In [23]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 125, 768])

In [27]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.einsum('ijk, ikl -> ijl', query, key.permute(0, 2, 1)) / math.sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    outputs = torch.bmm(weights, value)
    return outputs

# Multi-Head Attention

<figure style="text-align: center;">
  <img src="https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098136789/files/assets/nlpt_0305.png" alt="Operations in scaled dot-product attention" width="600" height="247">
  <figcaption>Multi-head attention (Figure 3-5, <em>Natural Language Processing with Transformers</em>)</figcaption>
</figure>

In [29]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim) -> None:
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)
    
    def forward(self, hidden_state):
        return scaled_dot_product_attention(self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))

In [30]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList([AttentionHead(embed_dim, head_dim) for _ in range(num_heads)])
        self.linear = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, hidden_state):
        x = torch.cat([head(hidden_state) for head in self.heads], dim=-1)
        return self.linear(x)

In [31]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.shape

torch.Size([1, 125, 768])

# Encoder Layer

In [32]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

In [35]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # Pre-layer layer norm setup
        hidden_state = self.layer_norm_1(x)
        x = x + self.attention(hidden_state)
        # Which can be merged into
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [36]:
encode_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encode_layer(inputs_embeds).shape

(torch.Size([1, 125, 768]), torch.Size([1, 125, 768]))

# Positional Embeddings

In [37]:
class Embeddings(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
    
    def forward(self, input_ids):
        # Create position IDs
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Create tokena nd position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        return self.dropout(self.layer_norm(embeddings))

In [38]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 125, 768])

In [39]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config) for _ in range(config.num_hidden_layers)])
    
    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [40]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).shape

torch.Size([1, 125, 768])

In [41]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, x):
        x = self.encoder(x)[:, 0, :]
        x = self.dropout(x)
        return self.classifier(x)
        

In [43]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids)

tensor([[0.2761, 0.7299, 0.5779]], grad_fn=<AddmmBackward0>)

In [46]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [47]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[27.2213,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
         [ 1.0387, 29.7964,    -inf,  ...,    -inf,    -inf,    -inf],
         [-1.2391,  0.4513, 29.2506,  ...,    -inf,    -inf,    -inf],
         ...,
         [ 0.1616, -0.7194, -1.5926,  ..., 27.7050,    -inf,    -inf],
         [-0.7834,  1.0085,  0.9046,  ...,  0.9289, 29.0313,    -inf],
         [ 0.5991, -0.1161,  1.2674,  ...,  0.4996, -0.5079, 28.2742]]],
       grad_fn=<MaskedFillBackward0>)

In [48]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.einsum('ijk, ikl -> ijl', query, key.permute(0, 2, 1)) / math.sqrt(dim_k)
    if mask is not None:
        scores.masked_fill(mask == 0, -float("inf"))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)